In [ ]:
from esper.prelude import *
from esper.spark import *
import pyspark.sql.functions as func

# Sum all faces

## Postgres

In [ ]:
from django.db.models import ExpressionWrapper
FaceGender.objects.annotate(
    duration=ExpressionWrapper(
        (F('face__shot__max_frame') - F('face__shot__min_frame'))/F('face__shot__video__fps'),
        models.FloatField())) \
    .aggregate(Sum('duration'))
# 5min 20s to completion

## BigQuery

In [ ]:
print("""
SELECT SUM(( `{t}_shot`.max_frame - `{t}_shot`.min_frame ) / `{t}_video`.fps ) AS duration 
FROM   `{t}_facegender`
       inner join `{t}_face` 
               ON ( `{t}_facegender`.face_id = `{t}_face`.id ) 
       left outer join `{t}_shot`
                    ON ( `{t}_face`.shot_id = `{t}_shot`.id ) 
       left outer join `{t}_video`
                    ON ( `{t}_shot`.video_id = `{t}_video`.id )
""".format(t="visualdb-1046.tvnews.query"))
# 10.2s to completion

## Spark SQL

In [ ]:
video = spark.table.query_video
facegender = spark.table.query_facegender
shot = spark.table.query_shot
face = spark.table.query_face

full_df = facegender \
    .join(face, facegender.face_id == face.id) \
    .join(shot, face.shot_id == shot.id) \
    .join(video, shot.video_id == video.id) \
    .select("*", ((shot.max_frame - shot.min_frame) / video.fps).alias('duration'))
    
full_df.agg(func.sum(full_df.duration)).collect()
# 15.5s to completion

In [ ]:
full_df \
    .where(video.show_id == Show.objects.get(name='Tucker Carlson Tonight').id) \
    .where(facegender.gender_id == Gender.objects.get(name='M').id) \
    .agg(func.sum(full_df.duration)).collect()

# Group by show and gender

## PostgreSQL

In [ ]:
# ??? how to even express this in the ORM?

## BigQuery

In [ ]:
print("""
SELECT `{t}_video`.show_id, `{t}_facegender`.gender_id, 
       SUM(( `{t}_shot`.max_frame - `{t}_shot`.min_frame ) / `{t}_video`.fps ) AS duration 
FROM   `{t}_facegender`
       inner join `{t}_face` 
               ON ( `{t}_facegender`.face_id = `{t}_face`.id ) 
       left outer join `{t}_shot`
                    ON ( `{t}_face`.shot_id = `{t}_shot`.id ) 
       left outer join `{t}_video`
                    ON ( `{t}_shot`.video_id = `{t}_video`.id )
GROUP BY `{t}_video`.show_id, `{t}_facegender`.gender_id       
""".format(t="visualdb-1046.tvnews.query"))
# 13.2s to completion

## Spark SQL

In [ ]:
full_df.groupBy(video.show_id, facegender.gender_id).agg(func.sum(full_df.duration)).collect()